In [16]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime
import pandas as pd
import requests
import json

In [17]:
tweets= []
for line in open('boulder_flood_geolocated_tweets.json','r',encoding="utf8"):
    tweets.append(json.loads(line))

In [18]:
clean_tweets =[]
for tweet in tweets:
    clean_tweets.append([tweet['created_at'],
                        tweet['text'],
                        tweet['coordinates']])

In [19]:
import csv
df = pd.DataFrame(clean_tweets, columns=['created_at','text','coordinates'])

In [20]:
df

,created_at,text,coordinates
0,Tue Dec 31 07:14:22 +0000 2013,Boom bitch get out the way! #drunk #islands #g...,"{'type': 'Point', 'coordinates': [-118.1004117..."
1,Tue Dec 31 18:49:31 +0000 2013,@WeatherDude17 Not that revved up yet due to m...,None
2,Mon Dec 30 20:29:20 +0000 2013,Story of my life! 😂 #boulder http://t.co/ZMfNK...,"{'type': 'Point', 'coordinates': [0.13429814, ..."
3,Mon Dec 30 23:02:29 +0000 2013,We're looking for the two who came to help a c...,None
4,Wed Jan 01 06:12:15 +0000 2014,Happy New Year #Boulder !!!! What are some of ...,"{'type': 'Point', 'coordinates': [144.98467167..."
...,...,...,...
18816,Thu Sep 12 04:12:20 +0000 2013,RT @EricRobertson: A more representative flood...,None
18817,Thu Sep 12 04:03:47 +0000 2013,Warning. Flash flood of boulder creek is immin...,None
18818,Thu Sep 12 04:11:56 +0000 2013,I just got an “Emergency Alert”! Nice to know ...,None
18819,Thu Sep 12 04:05:59 +0000 2013,RT @dailycamera: RT @CUBoulderPolice: Some fl...,None


In [43]:
df.to_csv("dsta.csv")

In [45]:
data=pd.read_csv("dsta.csv",encoding="latin-1")

In [46]:
data.head()

,Unnamed: 0,created_at,text,coordinates
0,0,Tue Dec 31 07:14:22 +0000 2013,Boom bitch get out the way! #drunk #islands #g...,"{'type': 'Point', 'coordinates': [-118.1004117..."
1,1,Tue Dec 31 18:49:31 +0000 2013,@WeatherDude17 Not that revved up yet due to m...,NaN
2,2,Mon Dec 30 20:29:20 +0000 2013,Story of my life! ð #boulder http://t.co/ZM...,"{'type': 'Point', 'coordinates': [0.13429814, ..."
3,3,Mon Dec 30 23:02:29 +0000 2013,We're looking for the two who came to help a c...,NaN
4,4,Wed Jan 01 06:12:15 +0000 2014,Happy New Year #Boulder !!!! What are some of ...,"{'type': 'Point', 'coordinates': [144.98467167..."


In [49]:
date_time=pd.to_datetime(df['created_at'])

In [54]:
df['created_at']=date_time

In [55]:
df.head()

,created_at,text,coordinates,date
0,2013-12-31 07:14:22+00:00,Boom bitch get out the way! #drunk #islands #g...,"{'type': 'Point', 'coordinates': [-118.1004117...",2013-12-31 07:14:22+00:00
1,2013-12-31 18:49:31+00:00,@WeatherDude17 Not that revved up yet due to m...,None,2013-12-31 18:49:31+00:00
2,2013-12-30 20:29:20+00:00,Story of my life! 😂 #boulder http://t.co/ZMfNK...,"{'type': 'Point', 'coordinates': [0.13429814, ...",2013-12-30 20:29:20+00:00
3,2013-12-30 23:02:29+00:00,We're looking for the two who came to help a c...,None,2013-12-30 23:02:29+00:00
4,2014-01-01 06:12:15+00:00,Happy New Year #Boulder !!!! What are some of ...,"{'type': 'Point', 'coordinates': [144.98467167...",2014-01-01 06:12:15+00:00


In [56]:
# mapping
ES = Elasticsearch(hosts=["localhost"], http_compress=True)
index_tweet = 'tweets'
mapping = {
    "settings": {
        "number_of_shards": "1",
        "number_of_replicas": "1"
        
      },
    
    
    "mappings": {
        "properties": {
            "created_at": { 'type': 'date'},
            "text": { 'type': 'text'},
            "coordinates": { 'type': 'geo_shape'} 
        }
    }
}
if ES.indices.exists(index_tweet):
    ES.indices.delete(index=index_tweet, ignore = 400)
ES.indices.create(index=index_tweet, body=mapping) 

<ipython-input-56-b3b560788a31>:20: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if ES.indices.exists(index_tweet):
C:\Users\Msys\anaconda3\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-56-b3b560788a31>:22: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  ES.indices.create(index=in

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

In [57]:
tweets=df.values.tolist()

In [58]:
tweets


[[Timestamp('2013-12-31 07:14:22+0000', tz='UTC'),
  'Boom bitch get out the way! #drunk #islands #girlsnight  #BJs #hookah #zephyrs #boulder #marines… http://t.co/uYmu7c4o0x',
  {'type': 'Point', 'coordinates': [-118.10041174, 34.14628356]},
  Timestamp('2013-12-31 07:14:22+0000', tz='UTC')],
 [Timestamp('2013-12-31 18:49:31+0000', tz='UTC'),
  '@WeatherDude17 Not that revved up yet due to model inconsistency. I\'d say 0-2" w/ a decent chance of &gt;1" #snow #COwx #weather #Denver',
  None,
  Timestamp('2013-12-31 18:49:31+0000', tz='UTC')],
 [Timestamp('2013-12-30 20:29:20+0000', tz='UTC'),
  'Story of my life! 😂 #boulder http://t.co/ZMfNKEl0xD',
  {'type': 'Point', 'coordinates': [0.13429814, 52.22500698]},
  Timestamp('2013-12-30 20:29:20+0000', tz='UTC')],
 [Timestamp('2013-12-30 23:02:29+0000', tz='UTC'),
  "We're looking for the two who came to help a cyclist after a hit-and-run at 30th/Baseline ~11pm Dec 23rd #Boulder #CO http://t.co/zyk3FkB4og",
  None,
  Timestamp('2013-12-30

In [59]:
# to insert  tweets 
for i in tweets:
    Tweets = {
        "created_at": i[0],
        "text": i[1],
        "coordinates": i[2]
    }
    